In [1]:
import nltk 
nltk.download("wordnet")
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from google.colab import drive
drive.mount('/drive', force_remount=True)

%cd "/drive/MyDrive/TextMiningProject/"

Mounted at /drive
/drive/MyDrive/TextMiningProject


In [3]:
!pip3 install --upgrade pandas

     |████████████████████████████████| 11.3 MB 14.0 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.5 which is incompatible.


In [3]:
!pip install tweet-preprocessor
!pip install sklearn
!pip install matplotlib
!pip install import_ipynb

!pip install pandas

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=1345836093e724534827a98d15c240246d8015a6cb22758b583bbb45639781b0
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb


In [4]:
import sys
import os

py_file_location = "/drive/MyDrive/text-preprocessing-techniques"
sys.path.append(os.path.abspath(py_file_location))

In [5]:
import pandas as pd
import random
import seaborn as sn
from sklearn.utils import shuffle
import numpy as np
from sklearn.model_selection import GridSearchCV
#import preprocessor
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, plot_confusion_matrix, confusion_matrix
from matplotlib import pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.dummy import DummyClassifier
import import_ipynb
from Utilities import *
from preprocess import tokenize 

In [6]:
genuine, ssb2, ssb3 = generate_data()


# genuine = pd.read_csv("tokenized_genuine.txt", header=None).rename({0:"text"}, axis=1)
# ssb2 = pd.read_csv("tokenized_ssb2.txt", header=None).rename({0:"text"}, axis=1)
# ssb3 = pd.read_csv("tokenized_ssb3.txt", header=None).rename({0:"text"}, axis=1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


In [14]:
train_1, train_2, test_1, test_2 = generate_train_test(genuine, ssb2, ssb3)


/drive/MyDrive/TextMiningProject/Utilities.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ssb2_train["Label"] = "bot"
/drive/MyDrive/TextMiningProject/Utilities.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ssb3_train["Label"] = "bot"
/drive/MyDrive/TextMiningProject/Utilities.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

In [15]:
def downsample_prop_ix(data_df, prop=0.5):
  count_trim = round(prop*len(data_df))
  
  df = data_df.sample(n=count_trim, random_state=1234)
  ix = list(df.index)

  return df, ix

In [16]:
## SSB2 Bot Tweets	

# test
ssb2_test = ssb2.sample(frac=0.3, replace=False, random_state=1234)
ssb2_test["Label"] = pd.Series(["bot" for i in range(len(ssb2_test))])
index_ssb2_test = list(ssb2_test.index)

# train
ssb2_train = ssb2.loc[~ssb2.index.isin(index_ssb2_test)]
ssb2_train["Label"] = pd.Series(["bot" for i in range(len(ssb2_train))])

df, _ = downsample_prop_ix(ssb2_train)
df, _ = downsample_prop_ix(df)
df, _ = downsample_prop_ix(df)
_, ix = downsample_prop_ix(df)

index_no_test = ix + index_ssb2_test

test_ssb2_true = ssb2.loc[~ssb2.index.isin(index_no_test)]

test_ssb2_true = downsample_prop(downsample_prop(downsample_prop(test_ssb2_true)))  
test_ssb2_true
#test_ssb2_true.to_csv('test_ssb2_true.csv')
test_ssb2_true.to_csv('test_ssb2_true_no_token.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [17]:
## SSB3 Bot Tweets	

# test	
ssb3_test = ssb3.sample(frac=0.3, replace=False, random_state=1234)
ssb3_test["Label"] = pd.Series(["bot" for i in range(len(ssb3_test))])
index_ssb3_test = list(ssb3_test.index)


# train
ssb3_train = ssb3.loc[~ssb3.index.isin(index_ssb3_test)]
ssb3_train["Label"] = pd.Series(["bot" for i in range(len(ssb3_train))])

df, _ = downsample_prop_ix(ssb3_train)
df, _ = downsample_prop_ix(df)
df, _ = downsample_prop_ix(df)
_, ix = downsample_prop_ix(df)

index_no_test = ix + index_ssb3_test

test_ssb3_true = ssb3.loc[~ssb3.index.isin(index_no_test)]

test_ssb3_true = downsample_prop(downsample_prop(downsample_prop(test_ssb3_true)))  
test_ssb3_true
#test_ssb3_true.to_csv('test_ssb3_true.csv')

test_ssb3_true.to_csv('test_ssb3_true_no_token.csv')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [18]:
## Genuine Human Tweets

# test	
genuine_test = genuine.sample(frac=0.3, replace=False, random_state=1234)
genuine_test["Label"] = pd.Series(["genuine" for i in range(len(genuine_test))])
index_genuine_test = list(genuine_test.index)


# train
genuine_train = genuine.loc[~genuine.index.isin(index_genuine_test)]
genuine_train["Label"] = pd.Series(["genuine" for i in range(len(genuine_train))])  

df, _ = downsample_prop_ix(genuine_train)
df, _ = downsample_prop_ix(df)
df, _ = downsample_prop_ix(df)
_, ix = downsample_prop_ix(df)

index_no_test = ix + index_genuine_test

test_genuine_sample_true = genuine.loc[~genuine.index.isin(index_no_test)]

test_genuine_sample_true = downsample_prop(downsample_prop(downsample_prop(test_genuine_sample_true)))  
test_genuine_sample_true
#test_ssb3_true.to_csv('test_genuine_sample_true.csv')

test_ssb3_true.to_csv('test_genuine_sample_true_no_token.csv')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
